##<center>Классификация статей ria.ru по рубрикам</center>
<center>Емельянов Виталий, ФИВТ</center>

###1. Извлечение данных
В качестве источника обучающей выборки используем сайт РИА-Новости

In [16]:
import newspaper

rianews = newspaper.build("http://ria.ru/", memoize_articles=False, language='ru')
print rianews.size()

1010


Посмотрим, как библиотека newspaper справляется с парсингом статей

In [20]:
first_article = rianews.articles[5]
first_article.download()
first_article.parse()
print first_article.url
print
print first_article.text

http://ria.ru/incidents/20150206/1046276722.html

МОСКВА, 6 фев — РИА Новости. Правоохранители обнаружили в общежитии одного из московских ВУЗов тело убитого студента, сообщило столичное управление СК РФ в пятницу.

"По данным следствия, 5 февраля 2015 года, примерно в 5 часов 10 минут, в общежитии одного из институтов по Измайловскому проспекту в городе Москве обнаружено тело студента 1993 года рождения с признаками насильственной смерти в виде колото-резанных ранений в области головы и шеи", — говорится в сообщении.

Проводится проверка, возбуждено дело об убийстве.

В каком именно институте учился молодой человек, не уточняется, но, по данным СМИ, речь идет об МГТУ имени Баумана.

Ранее в ряде СМИ сообщалось, что прошлой ночью около общежития МГТУ им. Баумана на востоке Москвы студент заступился за девушку, к которой пристали неизвестные. Они нанесли студенту несколько ножевых ранений, после чего скрылись, молодой человек скончался на месте. По другой версии, студент сам начал конфл

Выкачаем первые 990 статей, извлечем из каждой текст и сохраним в папке ria/ в отдельный файл %N.txt

In [ ]:
counter = 0
for i in xrange(990):
    rianews.articles[i].download()
    rianews.articles[i].parse()
    filename = "ria/" + str(counter) + ".txt"
    counter += 1
    with open(filename, 'w+') as f:
        f.write(rianews.articles[i].url)
        f.write('\n')
        f.write(rianews.articles[i].text.encode('utf-8'))

###2. Построение обучающей выборки

Посмотрим на структуру url сайта ria.ru

In [1]:
import os
import numpy as np


with open('ria/1.txt', 'r') as f:
    data_in = f.read()
lines = data_in.split('\n')
url = lines[0]  
print url

with open('ria/132.txt', 'r') as f:
    data_in = f.read()
lines = data_in.split('\n')
url = lines[0]  
print url

with open('ria/493.txt', 'r') as f:
    data_in = f.read()
lines = data_in.split('\n')
url = lines[0]  
print url

http://ria.ru/culture/20151115/1101521140.html
http://ria.ru/tv_sport/20151113/1320087183.html
http://ria.ru/economy/20151115/1321054488.html#comments


Теперь можно предположить, что рубрика статьи - это то, что идет после "http://ria.ru/" до  первого "/".
Извлечем из выкачанного множества статей, рубрику и текст статьи, исходя из нашего предположения

In [2]:
import os
import urlparse


file_list = os.listdir('./ria')

news_list = []
for filename in file_list:
    with open('ria/' + filename, 'r') as f:
        data_in = f.read()
        lines = data_in.split('\n')
        url = lines[0]
        topic = urlparse.urlsplit(url).path.split('/')[1]
        text = '\n'.join(lines[1:])
        news_list.append({'topic': topic, 'text': text.decode('utf-8')})
print "Number of articles:", len(news_list)

Number of articles: 498


Заведем set из рубрик и посмотрим на них

In [3]:
topics = dict()
for i in xrange(len(news_list)):
    if news_list[i]['topic'] not in topics:
       topics[news_list[i]['topic']] = 1
    else:
        topics[news_list[i]['topic']] += 1
  
print "Number of topics:", len(topics)
print
print "Topics:"

for item in sorted(topics.items()):
    print item[0], item[1]

Number of topics: 45

Topics:
Tech_news 1
analytics 7
announce 74
arab_sy 1
asia 3
culture 1
docs 1
economy 82
games 2
gl 3
history_video 3
incidents 7
infografika 1
moscow 6
multimedia 2
opinions 1
photolents 3
politics 74
r_chronicles 1
religion 1
science 2
sochi2014_torchrelay 1
society 6
syria 1
syria_is 1
syria_mm 4
tags 3
titanic_interactive 1
tournament 17
trend 2
tv_culture 1
tv_defense_safety 3
tv_eco 3
tv_economy 1
tv_incidents 9
tv_politics 20
tv_science 2
tv_society 12
tv_sport 5
valdaiclub_anniversary_announcement 1
victorina 4
video 8
videoclub 3
vote 3
world 111


Видим, что наше предположение оказалось верным и у нас 45 рубрик.

Суффикс tv у рубрики говорит о том, что в статье находится видеоролик как это можно видеть на примере http://ria.ru/tv_culture/20151111/1318661856.html<br/>
Будем считать, что tv_rubric то же, что и rubric. 

In [4]:
for news in news_list:
    if news['topic'] == "tv_culture":
        news['topic'] = "culture"
    if news['topic'] == "tv_economy":
        news['topic'] = "economy"
    if news['topic'] == "tv_incidents":
        news['topic'] = "incidents"
    if news['topic'] == "tv_politics":
        news['topic'] = "politics"
    if news['topic'] == "tv_society":
        news['topic'] = "society"
    if news['topic'] == "tv_sport":
        news['topic'] = "sport"
    if news['topic'] == "tv_science":
        news['topic'] = "science"
    if news['topic'] == "tv_defense_safety":
        news['topic'] = "defense_safety"
    if news['topic'] == "tv_eco":
        news['topic'] = "eco"

Также мы видим что есть 3 рубрики по Сирии: syria, syria_is, syria_mm.<br/>
Видимо, syria_is - все что касается ISIS, а syria_mm - military map. Объединим эти три рубрики в одну.

In [5]:
for news in news_list:
    if news['topic'] == "syria_is":
        news['topic'] = "syria"
    if news['topic'] == "syria_mm":
        news['topic'] = "syria"
        
topics = dict()
for i in xrange(len(news_list)):
    if news_list[i]['topic'] not in topics:
       topics[news_list[i]['topic']] = 1
    else:
        topics[news_list[i]['topic']] += 1
for item in sorted(topics.items()):
    print item[0], item[1]

Tech_news 1
analytics 7
announce 74
arab_sy 1
asia 3
culture 2
defense_safety 3
docs 1
eco 3
economy 83
games 2
gl 3
history_video 3
incidents 16
infografika 1
moscow 6
multimedia 2
opinions 1
photolents 3
politics 94
r_chronicles 1
religion 1
science 4
sochi2014_torchrelay 1
society 18
sport 5
syria 6
tags 3
titanic_interactive 1
tournament 17
trend 2
valdaiclub_anniversary_announcement 1
victorina 4
video 8
videoclub 3
vote 3
world 111


Исключим статьи и рубрики, где меньше 4 статей в рубрике.

In [6]:
min_size = 4

updated_news_list = []
updated_topics = dict()
for topic in topics:
    if topics[topic] >= min_size:
        updated_topics[topic] = topics[topic]
        for article in news_list:
            if article['topic'] == topic:
                updated_news_list.append({'topic': article['topic'], 'text': article['text']})
news_list = updated_news_list
topics = updated_topics

for item in sorted(topics.items()):
    print item[0], item[1]

analytics 7
announce 74
economy 83
incidents 16
moscow 6
politics 94
science 4
society 18
sport 5
syria 6
tournament 17
victorina 4
video 8
world 111


In [7]:
print "Number of articles: ", len(news_list)

Number of articles:  453


Теперь у нас есть размеченная выборка размером в 453 статьи.

###3. Построение признаков и baseline

Для построения baseline постороим bag-of-words.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

texts = []
answers = []
for article in news_list:
    answers.append(article['topic'])
    texts.append(article['text'])
 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

Применим какие-нибудь алгоритмы классификации к нашему bag-of-words, посмотрим на качество и выберем в качестве baseline алгоритм с лучшим качеством.

In [9]:
from sklearn.cross_validation import cross_val_score

In [10]:
from sklearn.naive_bayes import MultinomialNB

nb_classifier = MultinomialNB()
print "Quality: ", np.mean(cross_val_score(nb_classifier, X, np.array(answers), scoring="f1_weighted"))

Quality:  0.687415595575


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(n_estimators = 60)
print "Quality: ", np.mean(cross_val_score(random_forest_classifier, X, np.array(answers), scoring="f1_weighted"))

Quality:  0.700818312857


In [12]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression

logistic_classifier = LogisticRegression()
print "Quality: ", np.mean(cross_val_score(logistic_classifier, X, np.array(answers), scoring="f1_weighted"))

Quality:  0.759952584239


Логистическая регрессия показала самое лучшее качество. Посмотрим как она классифицирует следующую статью:

In [13]:
logistic_classifier.fit(X, answers)
ans = logistic_classifier.predict(vectorizer.transform(["Выход фильма «Сделано во Франции» о подготовке теракта в Париже отложили на неопределенное время, сообщает The Time. Такое решение кинопрокатчики приняли на фоне террористических атак, которые произошли во французской столице в ночь на 14 ноября."]))
print "Answer: ", ans[0]

Answer:  politics


Кажется, что это больше походит на анонс нежели на статью о политике. Хотя кто-то, думаю, может отнести ее и к политической новости.

###4. Более хитрые признаки 

В качестве признаков добавим еще 2-, 3-, 4-граммы и посмотрим на качество классификации логистической регрессией:

In [14]:
from sklearn.feature_extraction.text import CountVectorizer 
ngram_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 4), min_df=1) 
X = ngram_vectorizer.fit_transform(texts) 

from sklearn import linear_model 


logistic_classifier = LogisticRegression()
print "Quality: ", np.mean(cross_val_score(logistic_classifier, X, np.array(answers), scoring="f1_weighted"))

Quality:  0.759346971953


Видим, что качество осталось тем же, что и при использовании bag-of-words.